In [53]:
# set working directory
import os
os.chdir("/home/aiops/zhuty/tinyllama")

In [3]:
from pathlib import Path
import transformers
from transformers import AutoModel, AutoTokenizer, LlamaForCausalLM
from lit_gpt import Tokenizer

model = LlamaForCausalLM.from_pretrained("/home/aiops/zhuty/tinyllama/out/tinyllama_120M")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
## Generate text using this mdoel
from transformers import pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
text = "(Shandaken, NY) -- A New York City music teacher who had been missing for over a week is dead after he was found in the woods in Ulster County.\nForty-six-year-old Keith Johnson was last seen May 4th at his Queens school. His vehicle was found at the Woodland Valley Trailhead. Johnson, an avid hiker, was found suffering from hypothermia in the woods of Shandaken on Saturday. Officials say he died before medics were able to get to him. Foul play is not suspected."
generator(text, max_length=64, num_return_sequences=1, min_new_tokens=12)

ModuleNotFoundError: No module named 'lightning_utilities'

## Analyze the generated results

In [41]:
import json

# result_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/generated_queries/tinyllama_120M_20b.json"
# inputs_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/generated_queries/tinyllama_120M_20b_inputs.json"
result_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/generated_queries/tinyllama_120M.json"
inputs_path = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/generated_queries/tinyllama_120M_inputs.json"

In [42]:
with open(result_path, 'r') as f:
    results = json.load(f)
with open(inputs_path, 'r') as f:
    inputs = json.load(f)

In [47]:
for i in range(30):
    print("=====================================")
    print("Input: ", inputs['text'][i], "\n")
    print("Generated: ", results[i][0]['generated_text'])


Input:  aid. "What resonated with a lot of people was the Google ethos of 'do no evil' and doing tech for good." For talent pros seeking solutions, hearing how workers' causes connect to the organizational mission might help start conversations. Urging other leaders to address the root of the unrest may be a step toward fixing tech's brand of turnover, burnout and disengagement. "I always wanted to sell a product that people wanted to buy as a recruiter," Degg said. "Life is too short to hate your job." 

Generated:  
Designing for a Lead
"If you had a beast, that beast would be very self-adjusted, and there's nothing else you can do to help the process and produce it, you could have a built-in time to educate and encourage your employees to take time forward and understand the impact you have on them," Degg added.
Duty Supervisor
"I'm so grateful to have been
Input:  tant to try and reduce one’s stress levels, by following practices such as the ones suggested by CCOHS: take several de

In [51]:
jsonlines = []
# write to jsonlines
for i in range(len(results)):
    jsonlines.append({"id": inputs['id'][i],  "title": results[i][0]['generated_text']}) # title is for search

# # write to file
# with open("/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/generated_queries/chunk_0_1k.jsonl", 'w') as f:
#     for line in jsonlines:
#         f.write(json.dumps(line) + '\n')

In [56]:
from processing.graphs.utils import read_trec_results

In [61]:
new_result = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/bm25_search_results/chunk_0_gen1k.result.txt"
old_result = "/home/aiops/zhuty/ret_pretraining_data/redpajama_2b_id_added/bm25_search_results/chunk_0.result.txt"
new_result = read_trec_results(new_result)
old_result = read_trec_results(old_result)

In [64]:
def calculate_jaccard_at_k(list1, list2, k):
    set1 = set(item['doc_id'] for item in list1[:k])
    set2 = set(item['doc_id'] for item in list2[:k])
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def calculate_precision_at_k(list1, list2, k):
    set1 = set(item['doc_id'] for item in list1[:k])
    set2 = set(item['doc_id'] for item in list2[:k])
    intersection = len(set1.intersection(set2))
    return intersection / k

def compare_search_results_at_k(list1, list2, ks=[5, 10, 20]):
    results = {}
    for k in ks:
        jaccard_similarity = calculate_jaccard_at_k(list1, list2, k)
        precision_at_k = calculate_precision_at_k(list1, list2, k)
        results[k] = {
            "Jaccard Similarity at k": jaccard_similarity,
            "Precision at k": precision_at_k
        }
    return results

# # Example usage
# list1 = [...]  # Your first list
# list2 = [...]  # Your second list
#
# comparison_metrics = compare_search_results_at_k(list1, list2, ks=[5, 10, 20])
# for k, metrics in comparison_metrics.items():
#     print(f"Metrics at k={k}: {metrics}")


In [70]:
def calculate_average_metrics_at_k(all_results, ks=[5, 10, 20]):
    # Initialize dictionaries to store total metrics
    total_jaccard_at_k = {k: 0 for k in ks}
    total_precision_at_k = {k: 0 for k in ks}

    # Number of queries
    num_queries = len(all_results)

    for list1, list2 in all_results:
        for k in ks:
            jaccard_at_k = calculate_jaccard_at_k(list1, list2, k)
            precision_at_k = calculate_precision_at_k(list1, list2, k)

            total_jaccard_at_k[k] += jaccard_at_k
            total_precision_at_k[k] += precision_at_k

    # Calculate averages
    avg_jaccard_at_k = {k: total_jaccard_at_k[k] / num_queries for k in ks}
    avg_precision_at_k = {k: total_precision_at_k[k] / num_queries for k in ks}

    return avg_jaccard_at_k, avg_precision_at_k

# Example usage
# all_results is a list of tuples, each containing two lists (list1, list2) for each query
# all_results = [([...], [...]), ([...], [...]), ...]  # Replace [...] with actual lists for each query
all_results = [(new_result[i], old_result[i]) for i in new_result.keys()]

k_list = [1, 3, 5, 10, 20, 100]
avg_jaccard, avg_precision = calculate_average_metrics_at_k(all_results, ks=k_list)

for k in k_list:
    print(f"Average Jaccard Similarity at k={k}: {avg_jaccard[k]:.4f}")
    print(f"Average Precision at k={k}: {avg_precision[k]:.4f}")


Average Jaccard Similarity at k=1: 0.0180
Average Precision at k=1: 0.0180
Average Jaccard Similarity at k=3: 0.0208
Average Precision at k=3: 0.0334
Average Jaccard Similarity at k=5: 0.0245
Average Precision at k=5: 0.0412
Average Jaccard Similarity at k=10: 0.0324
Average Precision at k=10: 0.0560
Average Jaccard Similarity at k=20: 0.0402
Average Precision at k=20: 0.0700
Average Jaccard Similarity at k=100: 0.0676
Average Precision at k=100: 0.1168


In [6]:
# split the chunks
num_chunks = 4
shard_numbers = list(range(1, 89))
chunked_shard_numbers = [shard_numbers[i:i + num_chunks] for i in range(0, len(shard_numbers), num_chunks)]

for curr_chunk in chunked_shard_numbers:
    lower_bound = curr_chunk[0]
    upper_bound = curr_chunk[-1]
    print(f'sailctl job create bm25search{lower_bound}and{upper_bound} -p low -g 1  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama; bash processing/range_search_bm25.sh {lower_bound} {upper_bound} " --debug ; ')


sailctl job create bm25search1and4 -p low -g 1  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama; bash processing/range_search_bm25.sh 1 4 " --debug ; 
sailctl job create bm25search5and8 -p low -g 1  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama; bash processing/range_search_bm25.sh 5 8 " --debug ; 
sailctl job create bm25search9and12 -p low -g 1  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama; bash processing/range_search_bm25.sh 9 12 " --debug ; 
sailctl job create bm25search13and16 -p low -g 1  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama; bash processing/range_search_bm25.sh 13 16 " --debug ; 
sailctl job create bm25search17and20 -p low -g 1  --command-line-args "source /home/aiops/zhuty/start.sh ; cd /home/aiops/zhuty/tinyllama; bash processing/range_search_bm25.sh 17 20 " --debug ; 
sailctl job create bm25search21and2